# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Constants

In [2]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Settings

In [3]:
DetectorFactory.seed = RANDOM_STATE

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Code

### Load Dataset

In [5]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [6]:
len(df_reviews)

3212

In [7]:
# Höchstes Datum
max_date = df_reviews['date'].max()

# Niedrigstes Datum
min_date = df_reviews['date'].min()

print("Höchstes Datum:", max_date)
print("Niedrigstes Datum:", min_date)


Höchstes Datum: 2023-10-15
Niedrigstes Datum: 2022-10-15


### Filter Sentences

In [8]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [9]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,7a80890b-f6b2-4e31-9e3c-c0260c071f06
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,27c414cf-d356-4fbd-bac0-708b5951474f
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,5e0a4404-e9aa-45f6-b57c-fbd4c7f3988e
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,f7df217a-5847-418c-baf4-888f0f5f7a7b
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,edbae1fd-cec7-4378-ad7f-4e08924fafe3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,d3f906ab-3145-46e6-a349-c10a080c7c07
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,afb19676-500b-4b1b-b461-fbd292cf6b6f
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,198e163b-9412-4630-a734-66d69d6605b4
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,a09777f3-516c-4b6f-91e4-6716609fcc84


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [10]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
21,3,918788541,2005104,0,Nicht zu empfehlen! Zero Dienstleistungsverstä...,2023-09-28,cariocadagema74,"Berlin, Deutschland",Kein Cent!,1.0,berlin,Augustiner am Gendarmenmarkt,de,Dieser Laden macht den Ruhm Berlins als Servic...,10,b962321a-c45f-4d6a-872a-9d896ec74eb9,nl
54,9,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,325ff70d-ec45-4449-855f-3e4fab6829d4,es
103,18,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,97d01174-6092-460e-9390-c4d33c07646a,en
111,21,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,af8323a5-c279-497a-a7da-0ce72cd49781,id
115,22,883285244,947082,0,Ganz gut,2023-03-23,Mawenabert,NaN,Ansonsten alles gut,4.0,berlin,Burgermeister,de,Hatte einen Hausmeister Burger mit Pommes Baco...,2,ac4dc2fa-78f3-4dce-a702-a9a1fb1909ab,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13318,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Service super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,2,6b7f6f3d-d174-4b66-9483-80cf408e9575,en
13319,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Essen super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,3,37927044-3a79-4225-bcc8-8c146fa83a2a,ca
13322,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Klar!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,6,ce91a348-ed69-48ae-99c5-43a1af7b34c7,tr
13356,3195,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,Ein tolles Restaurant!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,0,a4956557-55b6-4f8d-b6f6-877a223d4aa5,fr


In [11]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

In [12]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,7a80890b-f6b2-4e31-9e3c-c0260c071f06,de
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,27c414cf-d356-4fbd-bac0-708b5951474f,de
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,5e0a4404-e9aa-45f6-b57c-fbd4c7f3988e,de
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,f7df217a-5847-418c-baf4-888f0f5f7a7b,de
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,edbae1fd-cec7-4378-ad7f-4e08924fafe3,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,d3f906ab-3145-46e6-a349-c10a080c7c07,de
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,afb19676-500b-4b1b-b461-fbd292cf6b6f,de
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,198e163b-9412-4630-a734-66d69d6605b4,de
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,a09777f3-516c-4b6f-91e4-6716609fcc84,de


### Balancing

In [13]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(800, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [14]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [15]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,864582525,2,1
1,864598719,0,1
2,864598719,1,1
3,864598719,2,1
4,864598719,3,1
...,...,...,...
3995,921389449,3,1
3996,921404419,4,1
3997,921404419,6,1
3998,921404419,8,1


### Alternate examples based on rating

In [16]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
3.0    800
5.0    800
2.0    800
4.0    800
Name: count, dtype: int64

In [17]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [18]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Dass man aber 5 Minuten warten muss, bis jeman...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,3,f0a7538c-3f56-49b9-913b-2c5a7079cae4,de
1,3048,912583402,1005252,0,Routinierte Langeweile,2023-08-24,P6002SFpetram,NaN,Die kostenpflichtige Schinkennummer wird auch ...,2.0,wiesbaden,BENNER´s Bistronomie,de,"Die Karte enthält keine Überraschung, der Serv...",1,f38259a0-a0d6-4dbe-90e6-1ae1ad8c9ccb,de
2,210,892418807,2271294,1,Enttäuscht aber noch ok,2023-05-29,Simi E,NaN,Unten war alles frei.,3.0,hamburg,Hard Rock Cafe,de,"Lange drauf gefreut, sehr enttäuscht. Obwohl M...",2,813d4c0f-5318-4c69-bfe8-01b5bb795d6c,de
3,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Auch für die anderen Begleiter war das Essen ok.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,5,c30d4932-388a-49bf-88bc-ca3cbfd7def2,de
4,2916,885457754,7322855,8,Perfekte Steaks und sehr aufmerksamer Service,2023-04-09,Pioneer63807824059,NaN,"Vielen Dank, wir kommen sehr gerne wieder !",5.0,augsburg,SteakManufaktur,de,Sehr leckeres Essen und toller Service von Tih...,1,ca63c88b-a679-478a-8e3a-13ddf5c84da7,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1141,880698268,984093,0,Speisekarte liest sich besser als serviertes E...,2023-03-02,C4049YQmelanies,NaN,und Convenience-Produkte.,1.0,düsseldorf,Im Füchschen,de,Im Füchschen hat man die Preise stark angehobe...,2,5da31017-77d5-4f8b-b7b8-b846af0a68db,de
3996,3129,864869547,6541566,1,Bewirtung Katastrophe!,2022-10-17,Simon E,NaN,"Essen war ok, aber sowas verdirbt einem nur di...",2.0,mönchengladbach,L'Osteria Mönchengladbach,de,Leider wurden wir gar nicht bedient. Wir haben...,4,48f9c295-f1a5-488f-8b98-e950bc0dd26b,de
3997,3015,880194942,2640161,0,Essen Flop👎 Ambiente/Service/Bier Top👍,2023-02-26,Vegasmann,"Rosenheim, Deutschland","Der Service war schnell, freundlich und Aufmer...",3.0,augsburg,Riegele Wirtshaus,de,Wir waren zum Abendessen im Riegele Wirtshaus....,1,395f3ce3-8953-49ab-9789-73296025a148,de
3998,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Das Ambiente ist natürlich besonders.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,6,08912473-cc39-4993-9aa8-faab203e1be1,de


### Store as .csv 

In [19]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [20]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [21]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Dass man aber 5 Minuten warten muss, bis jeman...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,3,f0a7538c-3f56-49b9-913b-2c5a7079cae4,de
1,3048,912583402,1005252,0,Routinierte Langeweile,2023-08-24,P6002SFpetram,NaN,Die kostenpflichtige Schinkennummer wird auch ...,2.0,wiesbaden,BENNER´s Bistronomie,de,"Die Karte enthält keine Überraschung, der Serv...",1,f38259a0-a0d6-4dbe-90e6-1ae1ad8c9ccb,de
2,210,892418807,2271294,1,Enttäuscht aber noch ok,2023-05-29,Simi E,NaN,Unten war alles frei.,3.0,hamburg,Hard Rock Cafe,de,"Lange drauf gefreut, sehr enttäuscht. Obwohl M...",2,813d4c0f-5318-4c69-bfe8-01b5bb795d6c,de
3,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Auch für die anderen Begleiter war das Essen ok.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,5,c30d4932-388a-49bf-88bc-ca3cbfd7def2,de
4,2916,885457754,7322855,8,Perfekte Steaks und sehr aufmerksamer Service,2023-04-09,Pioneer63807824059,NaN,"Vielen Dank, wir kommen sehr gerne wieder !",5.0,augsburg,SteakManufaktur,de,Sehr leckeres Essen und toller Service von Tih...,1,ca63c88b-a679-478a-8e3a-13ddf5c84da7,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1141,880698268,984093,0,Speisekarte liest sich besser als serviertes E...,2023-03-02,C4049YQmelanies,NaN,und Convenience-Produkte.,1.0,düsseldorf,Im Füchschen,de,Im Füchschen hat man die Preise stark angehobe...,2,5da31017-77d5-4f8b-b7b8-b846af0a68db,de
3996,3129,864869547,6541566,1,Bewirtung Katastrophe!,2022-10-17,Simon E,NaN,"Essen war ok, aber sowas verdirbt einem nur di...",2.0,mönchengladbach,L'Osteria Mönchengladbach,de,Leider wurden wir gar nicht bedient. Wir haben...,4,48f9c295-f1a5-488f-8b98-e950bc0dd26b,de
3997,3015,880194942,2640161,0,Essen Flop👎 Ambiente/Service/Bier Top👍,2023-02-26,Vegasmann,"Rosenheim, Deutschland","Der Service war schnell, freundlich und Aufmer...",3.0,augsburg,Riegele Wirtshaus,de,Wir waren zum Abendessen im Riegele Wirtshaus....,1,395f3ce3-8953-49ab-9789-73296025a148,de
3998,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Das Ambiente ist natürlich besonders.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,6,08912473-cc39-4993-9aa8-faab203e1be1,de


In [22]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64